In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !python3 -m pip install facenet-pytorch
# !python3 -m pip install face_alignment

In [3]:
!ls 'Videos7439' | wc -l

7439


In [4]:
!ls 'Spectrograms7439' | wc -l

7439


In [5]:
videos_folder = 'Videos7439'
spectrograms_folder = 'Spectrograms7439'

In [6]:
import cv2
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from facenet_pytorch import MTCNN
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import face_alignment
#import dlib
import requests
import csv
import itertools

/home1/riyaranj/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:

class GoogLeNetModified(models.GoogLeNet):
    def _transform_input(self, x):
        if x.size(1) == 1:
            # If the input has only one channel, replicate it to create three channels
            x = x.expand(-1, 3, -1, -1)

        x_ch0 = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
        x_ch1 = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
        x_ch2 = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
        x = torch.cat((x_ch0, x_ch1, x_ch2), 1)

        return x
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = GoogLeNetModified()
        # Modify the first layer to accept 3 channel input (for RGB images)
        self.features.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Modify the final layer to output the desired feature size
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)

        # Assuming the output is a tensor inside the GoogLeNetOutputs object
        logits_tensor = x.logits if hasattr(x, 'logits') else x  # Adjust accordingly based on the structure

        # Apply softmax directly on the logits tensor
        x_softmax = torch.nn.functional.softmax(logits_tensor, dim=1)
        return x_softmax






In [8]:
def extract_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    mid_frame_index = frame_count // 2  # Index of the frame in the middle of the video
    cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame_index)
    ret, frame = cap.read()
    if ret:
        cap.release()
        return frame
    else:
        cap.release()
        return None

In [9]:
def detect_face(frame):
    mtcnn = MTCNN()
    boxes, _ = mtcnn.detect(frame)
    if boxes is not None:
        # Assuming only one face in the frame
        box = boxes[0]
        x1, y1, x2, y2 = box
        # Crop the frame to the detected face
        cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
        return cropped_frame
    else:
        return None

In [10]:
# Function to download the pretrained face alignment model if it doesn't exist
def download_face_alignment_model(url, save_path):
    if not os.path.exists(save_path):
        print("Downloading pretrained face alignment model...")
        response = requests.get(url)
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print("Download complete.")

# Specify the URL of the pretrained face alignment model
face_alignment_model_url = "https://github.com/1adrianb/face-alignment-models/releases/download/2.0.1/2DFAN4-11f355bf06.pth.tar"

# Download the pretrained face alignment model if it doesn't exist
face_alignment_model_path = os.path.abspath("2DFAN4-11f355bf06.pth.tar")
download_face_alignment_model(face_alignment_model_url, face_alignment_model_path)

# Initialize face alignment model
fa = face_alignment.FaceAlignment(2, flip_input=False, device='cpu')  # 2 corresponds to 2D landmarks

def align_face(frame):
    # Perform face alignment
    aligned_faces = fa.get_landmarks(frame)
    if aligned_faces is not None:
        aligned_face = aligned_faces[0]  # Assuming only one face in the frame
        return aligned_face
    else:
        return None


In [11]:
def preprocess_image(frame):
    frame_pil = Image.fromarray(frame.astype('uint8'))
    frame_pil = frame_pil.convert('RGB')  # Convert to RGB format
    frame_np = np.array(frame_pil)
    print(frame_np.shape)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize using ImageNet mean and std
    ])
    frame_tensor = transform(frame_pil)
    return frame_tensor


In [12]:
def preprocess_spectrogram(image_path):
    img = Image.open(image_path).convert('RGB')  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match ViT input size
        transforms.ToTensor(),           # Convert to tensor
    ])
    img_tensor = transform(img)
    return img_tensor

In [13]:
def load_dataset(videos_folder):
    X = []
    y = []
    skipped_files = []
    video_files = [file for file in sorted(os.listdir(videos_folder)) if file.endswith(".flv")]
    for video_file in tqdm(video_files):
        video_path = os.path.join(videos_folder, video_file)
        frame = extract_frame(video_path)
        if frame is not None:
            cropped_face = detect_face(frame)
            if cropped_face is not None:
                preprocessed_face = preprocess_image(cropped_face)
                X.append(preprocessed_face)
                label = video_file.split("_")[2].split(".")[0]  # Adjusted to handle different file extensions
                if label == "HAP":
                    y.append(0)
                elif label == "SAD":
                    y.append(1)
                elif label == "ANG":
                    y.append(2)
                elif label == "DIS":
                    y.append(3)
                elif label == "FEA":
                    y.append(4)
                elif label == "NEU":
                    y.append(5)
            else:
                print(f"No face detected in {video_file}. Skipping.")
                skipped_files.append(video_file[:-3])
        else:
            print(f"Failed to extract frame from {video_file}. Skipping.")
            skipped_files.append(video_file[:-3])
    return X, y, skipped_files

In [14]:
def load_spectrogram_dataset(spectrograms_folder, skipped_files):
    X = []
    y = []
    # List all files in the input folder
    files = sorted(os.listdir(spectrograms_folder))
    # Iterate over files in the folder
    for filename in tqdm(files):
        if filename.endswith(".png") and filename[:-3] not in skipped_files:  # Assuming mel spectrograms are stored as PNG files
            input_path = os.path.join(spectrograms_folder, filename)
            img_tensor = preprocess_spectrogram(input_path)
            X.append(img_tensor)
            # Extract label from filename (assuming filename is in format "abc_IEO_label_xyz.png")
            label = filename.split("_")[2]
            if label == "HAP":
                y.append(0)
            elif label == "SAD":
                y.append(1)
            elif label == "ANG":
                y.append(2)
            elif label == "DIS":
                y.append(3)
            elif label == "FEA":
                y.append(4)
            elif label == "NEU":
                y.append(5)
    return X, y

In [15]:
# Define the ConcatDataset class to concatenate video frame and spectrogram tensors
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, X1, X2, y, modality='multimodal', presaved=False):
        self.X1 = X1
        self.X2 = X2
        self.y = y
        self.modality = modality
        self.presaved = presaved
    def __len__(self):
        if self.modality == 'audio':
          return len(self.X2)
        if self.modality == 'visual':
          return len(self.X1)
        return len(self.y)

    def __getitem__(self, idx):
        if not self.presaved:
          img1 = self.X1[idx]
          img2 = self.X2[idx]
          label = self.y[idx]
        else:
          img1 = torch.from_numpy(self.X1[idx]).float()  # Convert numpy array to torch tensor
          img2 = torch.from_numpy(self.X2[idx]).float()  # Convert numpy array to torch tensor
          label = torch.tensor(self.y[idx])  # Convert numpy array to torch tensor

        concatenated_img = torch.cat((img1, img2), dim=0)  # Concatenate along 0 dimension
        if self.modality == 'visual':
          return img1, label
        if self.modality == 'audio':
          return img2, label
        return concatenated_img, label # concatenate modalities

In [16]:
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [17]:
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [18]:
assert([file[:-3] for file in sorted(os.listdir(videos_folder))] == [file[:-3] for file in sorted(os.listdir(spectrograms_folder))])

In [19]:
_fullscale = True # Run fullscale experiment?
_presaved = True # Use presaved data .npy files?

# Check if data folders exists
if not os.path.exists(videos_folder):
    print("Videos folder does not exist.")
    sys.exit(1)
if not os.path.exists(spectrograms_folder):
    print("Spectrograms folder does not exist.")
    sys.exit(1)



In [21]:
# Load dataset and split into train and test sets
if _presaved:
  X = np.load('/home1/riyaranj/riya/X.npy', mmap_mode='r')
  y = np.load('/home1/riyaranj/riya/y.npy', mmap_mode='r')
  X_spec = np.load('/home1/riyaranj/riya/X_spec.npy', mmap_mode='r')
  y_spec = np.load('/home1/riyaranj/riya/y_spec.npy', mmap_mode='r')
else:
  X, y, skipped_files = load_dataset(videos_folder)
  X_spec, y_spec = load_spectrogram_dataset(spectrograms_folder, skipped_files)

In [22]:
# # Save X, y, X_spec, y_spec
# np.save('X_7439.npy', np.array(X))
# np.save('y_7439.npy', np.array(y))
# np.save('X_spec_7439.npy', np.array(X_spec))
# np.save('y_spec_7439.npy', np.array(y_spec))
# !cp 'X_7439.npy' '/content/drive/MyDrive/csci535/models/'
# !cp 'y_7439.npy' '/content/drive/MyDrive/csci535/models/'
# !cp 'X_spec_7439.npy' '/content/drive/MyDrive/csci535/models/'
# !cp 'y_spec_7439.npy' '/content/drive/MyDrive/csci535/models/'

In [23]:
print(f"Total number of samples: {len(X)}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"Number of train samples (video): {len(X_train)}", f"Number of test samples: {len(X_test)}")
X_train_spec, X_test_spec, y_train_spec, y_test_spec = train_test_split(X_spec, y_spec, test_size=0.3, random_state=42)
print(f"Number of train samples (audio): {len(X_train_spec)}", f"Number of test samples: {len(X_test_spec)}")



Total number of samples: 7429
Number of train samples (video): 5200 Number of test samples: 2229
Number of train samples (audio): 5200 Number of test samples: 2229


In [24]:
assert(len(X_train) == len(X_train_spec) and len(X_test) == len(X_test_spec))

In [25]:
if __name__ == "__main__":
    num_epochs = 50 # Example values, adjust as needed
    batch_sizes = [16, 32, 64]  # Example values, adjust as needed
    learning_rates = [0.01, 0.001, 0.0001]

    best_model_info = {}
    results_file = 'Latest_GoogLeNet_audio_video_combined_fullscale_cnn.csv'

    with open(results_file, mode='w', newline='') as csvfile:
        fieldnames = ['Model', 'Batch Size', 'Learning Rate', 'Num Epochs',  'Train Loss', 'Train Accuracy', 'Test Loss', 'Test Accuracy']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for lr in learning_rates:
            for bs in batch_sizes:
                # Initialize the model
                model = CNN(num_classes=6)  
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                model.to(device)

                # Define loss function and optimizer
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                    # Concatenate datasets
                train_dataset = ConcatDataset(X_train, X_train_spec, y_train, presaved = _presaved)
                test_dataset = ConcatDataset(X_test, X_test_spec, y_test, presaved = _presaved)

                # Create data loaders
                train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True)
                test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bs)
                print(f"Batch size: {bs}", f"Num of Epochs: {num_epochs}", f"lr: {lr}")
                # Training loop
                for epoch in range(num_epochs):
                    print("Epoch " + str(epoch))
                    train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
                  # Calculate test loss and accuracy
                    test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
                    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

                # Write results to CSV
                model_info = {'Model': f'Model_lr{lr}_bs{bs}_epochs{num_epochs}', 'Batch Size': bs, 'Learning Rate': lr, 'Num Epochs': num_epochs,
                              'Train Loss': train_loss, 'Train Accuracy': train_accuracy,
                              'Test Loss': test_loss, 'Test Accuracy': test_accuracy}
                writer.writerow(model_info)

                model_name = f'Latest_GoogLeNet_audio_video_combined_fullscale_cnn{num_epochs}_{bs}_{lr}'
                torch.save(model.state_dict(), model_name)

                # Check if this model is the best so far
                if not best_model_info or test_accuracy > best_model_info['Test Accuracy']:
                    best_model_info = {'Model': model_info['Model'], 'Batch Size': bs, 'Learning Rate': lr,
                                       'Train Loss': train_loss, 'Train Accuracy': train_accuracy,
                                       'Test Loss': test_loss, 'Test Accuracy': test_accuracy}

    # Print best model
    print("\nBest Model:")
    for key, value in best_model_info.items():
        print(f"{key}: {value}")


/home1/riyaranj/.local/lib/python3.12/site-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Batch size: 16 Num of Epochs: 50 lr: 0.01
Epoch 0


100%|██████████| 140/140 [00:01<00:00, 92.90it/s] 


Epoch 1/50, Train Loss: 1.8667, Train Accuracy: 0.1685, Test Loss: 1.8721, Test Accuracy: 0.1714
Epoch 1


100%|██████████| 140/140 [00:01<00:00, 115.53it/s]


...

Epoch 48/50, Train Loss: 1.1195, Train Accuracy: 0.9263, Test Loss: 1.4525, Test Accuracy: 0.5810
Epoch 48


100%|██████████| 35/35 [00:00<00:00, 36.26it/s]


Epoch 49/50, Train Loss: 1.1161, Train Accuracy: 0.9317, Test Loss: 1.3813, Test Accuracy: 0.6599
Epoch 49


100%|██████████| 35/35 [00:00<00:00, 35.97it/s]


Epoch 50/50, Train Loss: 1.1069, Train Accuracy: 0.9390, Test Loss: 1.3678, Test Accuracy: 0.6712

Best Model:
Model: Model_lr0.0001_bs64_epochs50
Batch Size: 64
Learning Rate: 0.0001
Train Loss: 1.106932061268733
Train Accuracy: 0.9390384615384615
Test Loss: 1.3678176697726312
Test Accuracy: 0.6711529834006281


In [26]:
print("Fin")

Fin
